# Hello world SageMaker and LangChain

In [1]:
from langchain import SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler
import json
from langchain_core.prompts import PromptTemplate
from langchain_core.callbacks import StdOutCallbackHandler
from langchain.chains import LLMChain

In [2]:
endpoint_name = "jumpstart-dft-hf-llm-mixtral-8x7b-i-20240319-180246"

In [9]:
class MixtralHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def transform_input(self, prompt: str, model_kwargs: dict) -> bytes:
        input_str = json.dumps({"inputs": prompt, "parameters": model_kwargs})
        return input_str.encode("utf-8")
        
    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        # modify this lower line for the model, some of them send lists, others send JSON directly
        return response_json[0]["generated_text"]



In [20]:
template = """ You are an AI Assistant named Mixtral. You are funny, kind, concise, and witty. 
You will get a question from your customer below. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Don't be too predictable, mix up your answers.

Question: {question}

Helpful Answer:"""

prompt = PromptTemplate.from_template(template)

In [21]:


llm = SagemakerEndpoint(endpoint_name = endpoint_name,
                        region_name = 'us-west-2',
                        #hopefully this is 
                        model_kwargs = { "max_new_tokens": 256, "do_sample": True},
                        # streaming=True,
                        content_handler = MixtralHandler()
                        # callbacks = [StdOutCallbackHandler()]
)

# handler = StdOutCallbackHandler()

chain = prompt | llm


In [18]:
chain.invoke({"question": "Are you there Mixtral? It's me, Emily."})

" Absolutely, Emily! I'm here to assist you. Thanks for asking!"

In [22]:
chain.invoke({"question": "Great! Good to meet you again. Can you tell me a joke?"})

" Of course, I'd be happy to! Why don't scientists trust atoms? Because they make up everything! Hope that gave you a chuckle."